# 5.5 读写文件

### 5.5.1 加载和保存张量

对于单个张量，我们可以直接调用`load`和`save`函数分别读写它们。

In [4]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'data/x-file')

我们现在可以将存储在文件中的数据读回内存

In [5]:
x2 = torch.load('data/x-file')
x2

tensor([0, 1, 2, 3])

我们可以存储一个张量列表，然后把它们读回内存

In [6]:
y = torch.zeros(4)
torch.save([x, y], 'data/xy-file')
x2, y2 = torch.load('data/xy-file')
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以写入或读取从字符串映射到张量的字典。当我们需要读取或写入模型中的所有权重时，这很方便。

In [7]:
mydict = {'x':x, 'y':y}
torch.save(mydict, 'data/mydict')
mydict2 = torch.load('data/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 5.2.2 加载和保存模型参数

保存单个权重向量(或者其他张量确实有用)，但是如果我们想保存整个模型，并在以后加载它们，单独保存每个向量则会变得很麻烦。因此，深度学习框架提供了内置函数来保存和加载整个网络。需要注意的一个细节是，这将保存模型的参数而不是整个模型。例如，如果我们有一个3层多层感知机，我们需要单独指定架构，因为模型本身可以包含任意代码，所以模型很难序列化。因此，为了恢复模型，我们需要用代码生成架构，再从磁盘加载数据。

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
net.state_dict()

OrderedDict([('hidden.weight',
              tensor([[ 0.1414,  0.1141,  0.1411,  ..., -0.1938, -0.0972,  0.1779],
                      [ 0.1290,  0.2083,  0.1019,  ..., -0.0998,  0.0869, -0.1471],
                      [ 0.1041, -0.0460,  0.2004,  ..., -0.0145,  0.2082, -0.1052],
                      ...,
                      [-0.1081,  0.2127,  0.1712,  ...,  0.1011, -0.1256, -0.1603],
                      [ 0.2049, -0.1692, -0.2016,  ..., -0.0199,  0.2208,  0.0893],
                      [-0.1357,  0.0959, -0.0024,  ..., -0.0014, -0.0049, -0.0782]])),
             ('hidden.bias',
              tensor([ 0.0437,  0.0841, -0.0536, -0.2051, -0.1086,  0.1979,  0.1654, -0.0346,
                      -0.1985, -0.0431,  0.0850,  0.0443, -0.2171,  0.1685,  0.0631,  0.0295,
                       0.1304,  0.1132, -0.1349,  0.0810, -0.1759,  0.1449,  0.0276, -0.0083,
                       0.1072, -0.1034, -0.1048,  0.0414, -0.0287, -0.2133,  0.1733, -0.0046,
                      -0.0378,

我们将模型的参数存储在一个叫做"MLP.params"的文件中

In [10]:
torch.save(net.state_dict(), 'data/MLP.params')

为了恢复模型，我们实例化原始多层感知机模型的一个备份。我们这里不需要随机初始化模型参数，而是直接读取文件中存储的参数。

In [13]:
clone = MLP()
clone.load_state_dict(torch.load('data/MLP.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个模型具有相同的模型参数，在输入相同的X时，两个实例的计算结果应该相同。

In [14]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])